In [3]:
import pandas as pd 
import numpy as np
import os

In [ ]:
! pip3 install langchain_community tiktoken langchain-openai langchainhub chromadb langchain


In [ ]:
!pip3 install langchain-chroma


In [ ]:
!pip3 install --upgrade "chromadb<0.5.0"


In [ ]:
!pip3 install --upgrade langchain-chroma


In [ ]:
!pip3 install -U langchain_openai
!pip3 install faiss-cpu

In [ ]:
!pip3 install pypdf

In [4]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'
os.environ['LANGCHAIN_PROJECT']="REIA"

In [5]:
os.environ['OPENAI_API_KEY'] = 'sk-proj-I1Z6g5eMHRnlUFR4exUXT3BlbkFJuyWWTVp5qgIvz9Oxehvp'


In [26]:
from langchain_community.document_loaders import PyPDFLoader

pdfloader = PyPDFLoader('../dataset/powerwall-overview-welcome-guide.pdf')
pdfpages = pdfloader.load_and_split()

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings


In [28]:
mydocuments = pdfloader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
texts = text_splitter.split_documents(mydocuments)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts,embeddings)


In [29]:
db

In [30]:
#using langchain retriever 
retriever = db.as_retriever()
from langchain.agents.agent_toolkits import create_retriever_tool
tool = create_retriever_tool(
    retriever,
    "powerwall_overview_welcome_guide",
    "searches and returns documents regarding the powerwall system"
)
tools = [tool]


#wrapp llm with conversational agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

#langchain given API to interact with chat models
from langchain_openai.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)
agent_executor = create_conversational_retrieval_agent(llm,tools,verbose=True)


In [32]:
input = "what is whole home backup"
result = agent_executor.invoke({"input":input})



> Entering new AgentExecutor chain...
Whole home backup refers to a configuration where the entire home is backed up by a recommended number of Powerwalls in the Powerwall system. In this setup, the Powerwalls provide backup power to all the electrical loads in the home during a grid outage. Depending on the energy requirements of the home, an upgrade to the electrical panel may be necessary to support the whole home backup system. Additional Powerwalls can be added to extend the duration of backup power and keep the house operational for a longer period during an outage.

> Finished chain.
